In [ ]:
# Choose variant and machine type
VARIANT = '2b-it' #@param ['2b', '2b-it', '9b', '9b-it', '27b', '27b-it']
MACHINE_TYPE = 'cuda' #@param ['cuda', 'cpu']

CONFIG = VARIANT[:2]
if CONFIG == '2b':
  CONFIG = '2b-v2'

In [ ]:
# Choose seed and PAWS parameters
SEED = 42 # @param int the answer to life, the universe, and everything

USE_PAWS = True #@param bool
PAWS_PERCENTILE = 30 # @param int

paws_dict_path = "paws_dict/msmarco_paws_dict.json"# @param str

In [ ]:
import os
from google.colab import userdata # `userdata` is a Colab API.

os.environ["KAGGLE_USERNAME"] = 'ucheochuba'
os.environ["KAGGLE_KEY"] = '5f5dd974f13fe89c391f02b0cf74e892'

In [ ]:
!pip install -q -U torch immutabledict sentencepiece

In [ ]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


In [ ]:
import torch
import os
import kagglehub
from datasets import load_dataset
from evaluate import load  # Use evaluate instead of load_metric


In [ ]:
# Download weights directory from Kaggle
try:
    weights_dir = kagglehub.model_download(f'google/gemma-2/pyTorch/gemma-2-{VARIANT}')
    print(f"Downloaded weights to: {weights_dir}")
except Exception as e:
    raise RuntimeError(f"Failed to download model weights: {e}")

Downloaded weights to: /kaggle/input/gemma-2/pytorch/gemma-2-2b-it/1


In [ ]:
 # clone in PAWS threshold dictionary
# Initialize a new git repository
!git init

# Add the remote repository
!git remote add origin https://github.com/ucheochuba/cs229s-final.git

# Enable sparse-checkout
!git config core.sparseCheckout true

# Set the folder path you want to checkout (e.g., paws_dict)
!echo "paws_dict/*" >> .git/info/sparse-checkout

# Pull the specific folder from the repository
!git pull origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 36 (delta 14), reused 31 (delta 9), pack-reused 0 (from 0)
Unpacking objects: 100% (36/36), 294.50 KiB | 1.45 MiB/s, done.
From https://github.com/ucheochuba/cs229s-final
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


In [ ]:
# NOTE: The "installation" is just cloning the repo.
# !git clone https://github.com/google/gemma_pytorch.git
!git clone https://github.com/ucheochuba/paws_gemma_pytorch gemma_pytorch

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 243 (delta 90), reused 62 (delta 56), pack-reused 116 (from 1)
Receiving objects: 100% (243/243), 2.18 MiB | 4.96 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [ ]:
import sys
sys.path.append('gemma_pytorch')

In [ ]:
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
from gemma.config import get_model_config
import kagglehub

In [ ]:
# Verify the presence of tokenizer and checkpoint files
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
if not os.path.isfile(tokenizer_path):
    raise FileNotFoundError(f"Tokenizer not found at: {tokenizer_path}")

ckpt_path = os.path.join(weights_dir, 'model.ckpt')
if not os.path.isfile(ckpt_path):
    raise FileNotFoundError(f"PyTorch checkpoint not found at: {ckpt_path}")

print("Tokenizer and checkpoint files verified.")

Tokenizer and checkpoint files verified.


### PAWS Stuff

In [ ]:
# load in files
import json

with open(paws_dict_path, 'r') as f:
    paws_dict = json.load(f)

In [ ]:
# Function to convert keys to integers recursively
def convert_keys_to_int(d):
    if isinstance(d, dict):
        # Create a new dictionary with integer keys
        return {int(k) if k.isdigit() else k: convert_keys_to_int(v) for k, v in d.items()}
    elif isinstance(d, list):
        # If the value is a list, recursively process each element
        return [convert_keys_to_int(item) for item in d]
    else:
        return d

# convert keys from str to int
paws_dict = convert_keys_to_int(paws_dict)

In [ ]:
# Load MS Marco v1.1 dataset
dataset = load_dataset("microsoft/ms_marco", "v1.1")

# Load Gemma 2B model
MODEL_VARIANT = "2b-v2"  # Update if needed
model_config = get_model_config(MODEL_VARIANT)
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Load tokenizer and model
# tokenizer = Tokenizer(tokenizer_path)
torch.set_default_dtype(model_config.get_dtype())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GemmaForCausalLM(model_config, use_paws=USE_PAWS, paws_percentile=PAWS_PERCENTILE, paws_dict=paws_dict)
model.load_weights(ckpt_path)
model.to(device).eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

Initializing attention layer 0 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 1 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 2 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 3 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 4 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 5 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 6 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 7 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 8 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 9 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 10 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 11 with PAWS activated. Cutoff percentile: 30.
Initializing attention layer 12 with PAWS activated. Cutoff percentile: 30.
Initializing attention

GemmaForCausalLM(
  (embedder): Embedding()
  (model): GemmaModel(
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): GemmaAttention(
          (qkv_proj): Linear()
          (o_proj): Linear()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear()
          (up_proj): Linear()
          (down_proj): Linear()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
        (pre_feedforward_layernorm): RMSNorm()
        (post_feedforward_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (sampler): Sampler()
)

In [ ]:
import random
from tqdm import tqdm

In [ ]:
#@title Metric Calculation functions

""" Official evaluation script for v1.1 of the SQuAD dataset. """

import argparse
import json
import re
import string
import sys
from collections import Counter


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    if len(prediction_tokens) == 0 and len(ground_truth_tokens) == 0: # model correctly didn't answer when there was no answer to give
      return 1
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def precision_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    if len(prediction_tokens) == 0 and len(ground_truth_tokens) == 0:
      return 1
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    return precision

def recall_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    if len(prediction_tokens) == 0 and len(ground_truth_tokens) == 0:
      return 1
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    recall = 1.0 * num_same / len(ground_truth_tokens)
    return recall

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def compute_score(predictions, references):
    f1 = exact_match = precision = recall = 0
    total = len(predictions)
    # print(predictions)
    # print(references)

    for pred, ref in zip(predictions, references):
        prediction_text = pred["prediction_text"]
        ground_truths = ref["answers"]["text"]

        exact_match += metric_max_over_ground_truths(exact_match_score, prediction_text, ground_truths)
        f1 += metric_max_over_ground_truths(f1_score, prediction_text, ground_truths)
        precision += metric_max_over_ground_truths(precision_score, prediction_text, ground_truths)
        recall += metric_max_over_ground_truths(recall_score, prediction_text, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    precision = 100.0 * precision / total
    recall = 100.0 * recall / total

    return {"exact_match": exact_match, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
#@title Helper functions
def format_prompt(context, query):
    """
    Format input for the Gemma model based on the updated tokenizer integration.
    """
    prompt = f"<start_of_turn>user\nAnswer as concisely as possible. Respond with only the requested information. Do not include periods, spaces, newlines, or any additional characters. Context: {context}\nQuestion: {query}<end_of_turn><eos>\n<start_of_turn>model\n"
    return prompt

def generate_answer(model, prompt, max_length=128, device="cuda"):
    """
    Generate an answer using the Gemma model with integrated tokenizer.
    """
    # Pass the prompt directly as a string
    outputs = model.generate(prompt, output_len=max_length, device=device)
    answer = outputs.split("<end_of_turn>")[0].split("<start_of_turn>model\n")[-1]
    return answer.strip()

def evaluate_marco(model, dataset, device="cuda"):
    """
    Evaluate Gemma on MS MARCO using Exact Match (EM), F1, Precision, and Recall metrics.
    """
    # metric = load("squad")
    # exact_match = 0
    # f1 = 0
    # total = len(dataset["validation"].select(range(3)))

    # Randomly sample 500 items from the dataset
    random.seed(SEED)
    val_data = dataset["validation"]
    sampled_indices = random.sample(range(len(val_data)), min(500, len(val_data)))
    small_val_data = val_data.select(sampled_indices)

    predictions = []
    references = []
    full_results = []

    for example in tqdm(small_val_data, desc="Evaluating MS MARCO on Small Validation Set"):
        context = example["passages"]['passage_text']
        query = example["query"]
        answers = example["answers"]
        if len(answers) == 0:
          answers = ['']

        # Generate model's answer
        prompt = format_prompt(context, query)
        prediction = generate_answer(model, prompt, max_length=128, device=device)

        # print("Context:", context)
        # print("Question:", question)
        # print("Expected Answers:", answers)
        # print("Model Prediction:", prediction)

        # Collect the data into a dictionary
        result = {
            "context": context,
            "query": query,
            "expected_answers": answers,
            "model_prediction": prediction
        }

        # Append the result to the list
        full_results.append(result)

        # Prepare predictions and references in the format expected by the metric
        predictions.append({
            "prediction_text": prediction,
            "id": str(len(predictions))  # Add a unique ID
        })

        references.append({
            "answers": {
                "text": answers,
                "answer_start": [0] * len(answers)  # Provide a default answer_start
            },
            "id": str(len(references))  # Add a unique ID
        })

    # Save the results to a JSON file
    path = f'percentile_{PAWS_PERCENTILE}_evaluation_results.json'
    try:
        with open(path, 'w') as f:
            json.dump(full_results, f, indent=4)
        print(f"Evaluation results saved to {path}")
    except Exception as e:
        print(f"Error saving results: {e}")


    # Compute metrics
    marco_scores = compute_score(predictions, references)

    # Calculate average scores
    exact_match = marco_scores["exact_match"]
    f1 = marco_scores["f1"]
    precision = marco_scores["precision"]
    recall = marco_scores["recall"]

    print(f"Exact Match (EM): {exact_match:.2f}%")
    print(f"F1 Score: {f1:.2f}%")
    print(f"Precision: {precision:.2f}%")
    print(f"Recall: {recall:.2f}%")

    return sampled_indices

In [ ]:
# Single validation example
example = dataset["validation"][0]
print(example)

context = example["passages"]['passage_text']
query = example["query"]
answers = example["answers"]

# Format prompt
prompt = format_prompt(context, query)

# Generate answer
prediction = generate_answer(model, prompt, max_length=128, device=device)

# Debug output
print("Context:", context)
print("Query:", query)
print("Expected Answers:", answers)
print("Model Prediction:", prediction)

{'answers': ['Approximately $15,000 per year.'], 'passages': {'is_selected': [1, 0, 0, 0, 0, 0], 'passage_text': ['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.', 'The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).', 'In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is a

In [ ]:
# Run evaluation
val_indices = evaluate_marco(model, dataset, device=device)

Evaluating MS MARCO on Small Validation Set: 100%|██████████| 500/500 [1:14:36<00:00,  8.95s/it]

Evaluation results saved to percentile_30_evaluation_results.json
Exact Match (EM): 0.00%
F1 Score: 6.58%
Precision: 6.09%
Recall: 14.04%


In [ ]:
# retrieve json from disk
from google.colab import files
files.download(f'percentile_{PAWS_PERCENTILE}_evaluation_results.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# random.seed(SEED)
# val_data = dataset["validation"]
# sampled_indices = random.sample(range(len(val_data)), min(500, len(val_data)))
# small_val_data = val_data.select(sampled_indices)

# predictions = []
# references = []
# full_results = []

# # for example in dataset["validation"].select(range(3)):
# empty = 0
# for example in tqdm(small_val_data, desc="Evaluating MS MARCO on Small Validation Set"):
#     # context = example["passages"]['passage_text']
#     # query = example["query"]
#     if len(example['answers']) == 0:
#       print(example)
#       empty += 1